In [ ]:
# !pip install pmdarima

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
import pmdarima as pm

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
ad = "linclin.csv"
adg = '/content/linclin.csv'

dfr = pd.read_csv(ad, index_col = "Month")

In [ ]:
# dfr = df.T
# # dfr.rename(columns={"Region_Name":"Month"}, inplace=True)
# dfr.insert(0, "Months", np.arange(1, 73))
# dfr.reset_index(inplace=True)
# dfr.drop(columns=["index"], inplace=True)

In [ ]:
dfr["Region 69"]

In [ ]:
mon = ["Month {}".format(i) for i in range(73, 88)]
dict = {"Month":mon}
fincsv = pd.DataFrame(dict)

In [ ]:
for i in range(1, 2):
  print(i)
  region = "Region " + str(i)
  Ntest = 15
  train = dfr.iloc[:, i]
  # test = dfr.iloc[-Ntest:, i]
  train_idx = dfr[region].index <= train.index[-1]
  # test_idx = dfr[region].index > train.index[-1]
  model = pm.auto_arima(train,
                        trace=True,
                        suppress_warnings=True,
                        max_p=12, max_q=6, max_order=20,
                        seasonal=True,step_wise=False, m=12)
  train_pred = model.predict_in_sample(start=12, end=-1)
  test_pred, confint = model.predict(n_periods=Ntest, return_conf_int=True)
  fincsv["Region {}".format(i)] = test_pred

  fig, ax = plt.subplots(figsize=(10, 5))
  ax.plot(dfr[region].index, dfr[region], label='data')
  ax.plot(np.arange(13, 73), train_pred, label='fitted')
  ax.plot(np.arange(72, 87), test_pred, label='forecast')
  ax.fill_between(np.arange(72, 87), \
                  confint[:,0], confint[:,1], \
                  color='red', alpha=0.3)
  ax.legend();

  test_pred.shape

In [ ]:
fincsv.set_index("Month")
fincsv.to_csv('autoarcsv.csv')
fincsv.iloc[:, -5:]